In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import copy
import multiprocessing as mp
from multiprocessing import freeze_support

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
batch_size = 200

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)
        # self.conv2 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1)
        # self.conv3 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1)
        # self.conv4 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1)
        # self.conv5 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1)
        # self.conv6 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        # self.fc1 = nn.Linear(512 * 4 * 4, 1024)
        # self.fc2 = nn.Linear(1024, 10)
        # self.fc3 = nn.Linear(84, 10)

        self.conv11 = nn.Conv2d(in_channels = 3, out_channels = 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv22 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1)
        self.fc11 = nn.Linear(512 * 8 * 8, 1200)
        self.fc22 = nn.Linear(1200, 10)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = F.relu(self.conv3(x))
        # x = self.pool(F.relu(self.conv4(x)))
        # x = F.relu(self.conv5(x))
        # x = self.pool(F.relu(self.conv6(x)))
        # x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        # x = F.relu(self.fc1(x))
        # # x = F.relu(self.fc2(x))
        # x = self.fc2(x)

        x = self.pool(F.relu(self.conv11(x)))
        x = self.pool(F.relu(self.conv22(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc11(x))
        x = self.fc22(x)

        return x


model = Net().to(device)
# net.cuda()

import time
import torch.nn as nn
import torch.optim as optim

epoch_set = []
train_loss_set = []
train_accuracy_set = []
test_loss_set = []
test_accuracy_set = []
test_floating_loss_set = []
test_floating_accuracy_set = []

# model = Net()
# model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)   #lr=0.01, momentum=0.9      lr=0.001, momentum=1

training_epochs = 100
start = time.time()

for epoch in range(1, 1+training_epochs):
    train(epoch)
    test()

print("time :", time.time() - start)

In [ ]:
# 14 bit Linearity split -0.1~0.1
def fc_chip1(copyWeight, copyGrad):
    ZeroW = torch.zeros_like(copyWeight)

    PotGrad = copy.deepcopy(copyGrad)
    PotGrad = torch.where(PotGrad < 0, 1., 0.)
    PotWeight = copy.deepcopy(copyWeight)
    UpperLim = 0.1*torch.ones_like(copyWeight)
    PotWeight = torch.where(PotGrad == 1, PotWeight + 1.22078e-5, ZeroW)    # Pot 0
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-6.10333E-5*PotWeight+1.32073E-5), ZeroW)    # Pot 1
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-1.22063E-4*PotWeight+1.60273E-5), ZeroW)    # Pot 2
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-1.83089E-4*PotWeight+2.02275E-5), ZeroW)    # Pot 3
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-2.44111E-4*PotWeight+2.5322E-5), ZeroW)     # Pot 4
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-3.05129E-4*PotWeight+3.09269E-5), ZeroW)    # Pot 5
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-3.66144E-4*PotWeight+3.67964E-5), ZeroW)    # Pot 6
    PotWeight = torch.where(PotWeight < 0.1, PotWeight, UpperLim)  # Pot clamping

    DepGrad = copy.deepcopy(copyGrad)
    DepGrad = torch.where(DepGrad > 0, 1., 0.)
    DepWeight = copy.deepcopy(copyWeight)
    LowerLim = -0.1*torch.ones_like(copyWeight)
    DepWeight = torch.where(DepGrad == 1, DepWeight - (1.22078e-5), ZeroW)    # Dep 0
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-6.10296E-5*DepWeight -1.32065E-5), ZeroW)    # Dep -1
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-1.22055E-4*DepWeight -1.60263E-5), ZeroW)    # Dep -2
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-1.83078E-4*DepWeight -2.02262E-5), ZeroW)    # Dep -3
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-2.44096E-4*DepWeight -2.53204E-5), ZeroW)    # Dep -4
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-3.05111E-4*DepWeight -3.0925E-5), ZeroW)     # Dep -5
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-3.66122E-4*DepWeight -3.67941E-5), ZeroW)    # Dep -6
    DepWeight = torch.where(DepWeight > -0.1, DepWeight, LowerLim)  # Dep clamping

    KeepWeight = copy.deepcopy(copyWeight)
    KeepWeight = torch.where(copyGrad == 0,  KeepWeight, ZeroW)

    weight = PotWeight + DepWeight + KeepWeight

    return weight

# 14 bit Linearity split -0.4~0.4
def fc_chip2(copyWeight, copyGrad):
    ZeroW = torch.zeros_like(copyWeight)

    PotGrad = copy.deepcopy(copyGrad)
    PotGrad = torch.where(PotGrad < 0, 1., 0.)
    PotWeight = copy.deepcopy(copyWeight)
    UpperLim = 0.4*torch.ones_like(copyWeight)
    PotWeight = torch.where(PotGrad == 1, PotWeight + 4.88311e-5, ZeroW)
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-6.10333E-5*PotWeight+5.28293E-5), ZeroW)    # Pot 1
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-1.22063E-4*PotWeight+6.41091E-5), ZeroW)    # Pot 2
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-1.83089E-4*PotWeight+8.09099E-5), ZeroW)    # Pot 3
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-2.44111E-4*PotWeight+1.01288E-4), ZeroW)    # Pot 4
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-3.05129E-4*PotWeight+1.23708E-4), ZeroW)    # Pot 5
    # PotWeight = torch.where(PotGrad == 1, PotWeight + (-3.66144E-4*PotWeight+1.47185E-4), ZeroW)    # Pot 6
    PotWeight = torch.where(PotWeight < 0.4, PotWeight, UpperLim)

    DepGrad = copy.deepcopy(copyGrad)
    DepGrad = torch.where(DepGrad > 0, 1., 0.)
    DepWeight = copy.deepcopy(copyWeight)
    LowerLim = -0.4*torch.ones_like(copyWeight)
    DepWeight = torch.where(DepGrad == 1, DepWeight - (4.88311e-5), ZeroW)
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-6.10296E-5*DepWeight -5.28261E-5), ZeroW)    # Dep -1
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-1.22055E-4*DepWeight -6.41052E-5), ZeroW)    # Dep -2
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-1.83078E-4*DepWeight -8.0905E-5), ZeroW)    # Dep -3
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-2.44096E-4*DepWeight -1.01282E-4), ZeroW)    # Dep -4
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-3.05111E-4*DepWeight -1.237E-4), ZeroW)    # Dep -5
    # DepWeight = torch.where(DepGrad == 1, DepWeight + (-3.66122E-4*DepWeight -1.47176E-4), ZeroW)    # Dep -6
    DepWeight = torch.where(DepWeight > -0.4, DepWeight, LowerLim)

    KeepWeight = copy.deepcopy(copyWeight)
    KeepWeight = torch.where(copyGrad == 0, KeepWeight, ZeroW)


    weight = PotWeight + DepWeight + KeepWeight

    return weight

In [ ]:
import time
import torch.nn as nn
import torch.optim as optim

epoch_set = []
train_loss_set = []
train_accuracy_set = []
test_loss_set = []
test_accuracy_set = []
test_floating_loss_set = []
test_floating_accuracy_set = []

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)   #lr=0.01, momentum=0.9      lr=0.001, momentum=1


model.conv11.weight.data = Convweight1
model.conv22.weight.data = Convweight2

model.fc11.weight.data = FCweight1
model.fc22.weight.data = FCweight2


training_epochs = 100
start = time.time()

for epoch in range(1, 1+training_epochs):
    train(epoch)
    test()

print("time :", time.time() - start)